In [18]:
# -*- coding: utf-8 -*-

import os
import json
import logging
from dotenv import load_dotenv
from openai import OpenAI, APIError, AuthenticationError, RateLimitError
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.shapes import MSO_SHAPE_TYPE # Placeholder 유형 확인에 사용될 수 있음
from pptx.enum.text import MSO_ANCHOR, MSO_AUTO_SIZE # 텍스트 프레임 속성 설정에 사용될 수 있음
from pptx.exc import PackageNotFoundError

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- 섹션 1: 환경 설정 및 라이브러리 임포트 ---

#.env 파일에서 환경 변수 로드
load_dotenv()

# OpenAI API 클라이언트 초기화
try:
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    if not client.api_key:
        raise ValueError("OpenAI API 키가.env 파일에 설정되지 않았거나 로드되지 않았습니다.")
except ValueError as e:
    logging.error(f"OpenAI 클라이언트 초기화 오류: {e}")
    exit(1) # API 키 없이는 진행 불가

# --- 섹션 2: OpenAI API 연동 함수 정의 ---

def generate_presentation_outline(prompt: str) -> dict | None:
    """
    사용자의 자연어 입력을 받아 프레젠테이션 개요(슬라이드 제목 목록 및 요약)를 생성합니다.
    OpenAI의 구조화된 출력(json_schema) 기능을 사용하여 안정적인 JSON 응답을 받습니다.

    Args:
        prompt (str): 사용자가 입력한 프레젠테이션 주제.

    Returns:
        dict | None: 생성된 프레젠테이션 개요 (JSON 형식) 또는 오류 발생 시 None.
    """
    logging.info(f"프레젠테이션 개요 생성 시작: 주제='{prompt}'")
    system_prompt = "You are an assistant skilled in creating presentation outlines."
    user_prompt = (
        f"Generate a presentation outline for the topic: '{prompt}'. "
        f"Provide a list of concise slide titles and a one-sentence summary for each slide. "
        f"The output must be a JSON object following the specified schema."
    )

    # OpenAI에 전달할 JSON 스키마 정의
    outline_schema = {
        "type": "object",
        "properties": {
            "slides": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "title": {"type": "string", "description": "Concise title for the slide"},
                        "summary": {"type": "string", "description": "One-sentence summary of the slide's content"}
                    },
                    "required": ["title", "summary"],
                    "additionalProperties": False # 스키마에 정의되지 않은 속성 허용 안 함
                }
            }
        },
        "required": ["slides"],
        "additionalProperties": False
    }

    try:
        response = client.chat.completions.create(
            model="gpt-4o", # 또는 사용 가능한 최신/적합 모델 [17, 18]
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            response_format={
                "type": "json_schema",
                "json_schema": {
                    "name": "presentation_outline",
                    "strict": True, # 스키마를 엄격하게 준수하도록 요청 [19, 20]
                    "schema": outline_schema
                }
            },
            temperature=0.7, # 창의성과 일관성 조절 (0.0 ~ 2.0)
        )
        
        # 응답 내용 확인 및 로깅
        response_content = response.choices[0].message.content
        logging.debug(f"OpenAI API 응답 (개요): {response_content}")

        # JSON 파싱
        outline_data = json.loads(response_content)
        logging.info("프레젠테이션 개요 생성 완료.")
        return outline_data

    except (APIError, AuthenticationError, RateLimitError) as e:
        logging.error(f"OpenAI API 오류 (개요 생성 중): {e}")
    except json.JSONDecodeError as e:
        logging.error(f"OpenAI 응답 JSON 파싱 오류 (개요): {e}\n응답 내용: {response_content}")
    except Exception as e:
        logging.error(f"예상치 못한 오류 발생 (개요 생성 중): {e}")

    return None

def generate_slide_content(slide_title: str, presentation_context: str) -> dict | None:
    """
    개별 슬라이드 제목과 전체 프레젠테이션 컨텍스트를 받아 상세 내용을 생성합니다.
    OpenAI의 구조화된 출력(json_schema) 기능을 사용합니다.

    Args:
        slide_title (str): 개요에서 가져온 슬라이드 제목.
        presentation_context (str): 전체 프레젠테이션 주제 또는 개요 요약.

    Returns:
        dict | None: 생성된 슬라이드 상세 내용 (JSON 형식) 또는 오류 발생 시 None.
    """
    logging.info(f"슬라이드 내용 생성 시작: 제목='{slide_title}'")
    system_prompt = "You are an assistant skilled in creating detailed presentation slide content."
    user_prompt = (
        f"Generate detailed content for a presentation slide titled '{slide_title}'. "
        f"The overall presentation topic is '{presentation_context}'. "
        f"Provide a refined slide title and 3-5 bullet points suitable for the slide. "
        f"The output must be a JSON object following the specified schema."
    )

    # 슬라이드 내용에 대한 JSON 스키마 정의
    slide_content_schema = {
        "type": "object",
        "properties": {
            "slide_title": {"type": "string", "description": "Refined title for the slide"},
            "bullet_points": {
                "type": "array",
                "items": {"type": "string"},
                "description": "List of 3-5 bullet points for the slide content"
            }
        },
        "required": ["slide_title", "bullet_points"],
        "additionalProperties": False
    }

    try:
        response = client.chat.completions.create(
            model="gpt-4o", # 또는 사용 가능한 최신/적합 모델
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            response_format={
                "type": "json_schema",
                "json_schema": {
                    "name": "slide_content",
                    "strict": True,
                    "schema": slide_content_schema
                }
            },
            temperature=0.7,
        )
        
        response_content = response.choices[0].message.content
        logging.debug(f"OpenAI API 응답 (슬라이드 내용): {response_content}")

        slide_data = json.loads(response_content)
        
        # 생성된 bullet points 개수 확인 (선택적)
        if not (3 <= len(slide_data.get('bullet_points',)) <= 5):
             logging.warning(f"'{slide_title}' 슬라이드의 글머리 기호 개수가 3-5개 범위를 벗어납니다: {len(slide_data.get('bullet_points',))}개")

        logging.info(f"슬라이드 내용 생성 완료: 제목='{slide_title}'")
        return slide_data

    except (APIError, AuthenticationError, RateLimitError) as e:
        logging.error(f"OpenAI API 오류 (슬라이드 내용 생성 중 - '{slide_title}'): {e}")
    except json.JSONDecodeError as e:
        logging.error(f"OpenAI 응답 JSON 파싱 오류 (슬라이드 내용 - '{slide_title}'): {e}\n응답 내용: {response_content}")
    except Exception as e:
        logging.error(f"예상치 못한 오류 발생 (슬라이드 내용 생성 중 - '{slide_title}'): {e}")

    return None

# --- 섹션 3: 파워포인트 생성 함수 정의 ---

def create_presentation_from_natural_language(user_prompt: str, template_path: str, output_filename: str):
    """
    자연어 입력을 받아 템플릿을 사용하여 파워포인트 프레젠테이션을 생성하는 메인 함수.

    Args:
        user_prompt (str): 사용자가 입력한 프레젠테이션 주제.
        template_path (str): 사용할 파워포인트 템플릿 파일 경로.
        output_filename (str): 저장할 프레젠테이션 파일 이름.
    """
    logging.info("프레젠테이션 생성 프로세스 시작.")

    # --- 단계 1: 개요 생성 ---
    presentation_outline = generate_presentation_outline(user_prompt)
    if not presentation_outline or 'slides' not in presentation_outline or not presentation_outline['slides']:
        logging.error("프레젠테이션 개요를 생성하지 못했거나 유효하지 않습니다. 프로세스를 중단합니다.")
        return

    # --- 단계 2: 템플릿 로드 ---
    try:
        prs = Presentation(template_path)
        logging.info(f"템플릿 로드 성공: {template_path}")
    except PackageNotFoundError:
        logging.error(f"템플릿 파일을 찾을 수 없습니다: {template_path}")
        return
    except Exception as e:
        logging.error(f"템플릿 파일 로드 중 오류 발생: {e}")
        return

    # --- 단계 3: 사용할 레이아웃 식별 ---
    #!!! 중요: 아래 인덱스는 예시이며, 사용자의 템플릿에 맞게 수정해야 합니다.!!!
    # 사용자의 템플릿을 파워포인트에서 열어 '보기 > 슬라이드 마스터'에서 레이아웃 순서와 이름을 확인하거나,
    # 아래 주석 처리된 코드를 사용하여 프로그래밍 방식으로 확인하세요.
    # --------------------------------------------------------------------------
    # 사용 가능한 레이아웃 이름과 인덱스 출력 (사용자 확인용)
    # print("템플릿의 사용 가능한 슬라이드 레이아웃:")
    # for i, layout in enumerate(prs.slide_layouts):
    #     print(f"  인덱스 {i}: {layout.name}")
    # --------------------------------------------------------------------------
    try:
        # 예시: 첫 번째 레이아웃을 제목 슬라이드로, 두 번째 레이아웃을 내용 슬라이드로 사용
        title_layout_index = 0
        content_layout_index = 1
        title_layout = prs.slide_layouts[title_layout_index]
        content_layout = prs.slide_layouts[content_layout_index]
        logging.info(f"사용할 레이아웃 식별: 제목({title_layout_index}), 내용({content_layout_index})")
    except IndexError:
        logging.error(f"템플릿에서 지정된 레이아웃 인덱스({title_layout_index} 또는 {content_layout_index})를 찾을 수 없습니다. "
                      f"템플릿에 최소 {max(title_layout_index, content_layout_index) + 1}개의 레이아웃이 있는지 확인하고 인덱스를 조정하세요.")
        return
    except Exception as e:
        logging.error(f"레이아웃 식별 중 오류 발생: {e}")
        return

    # --- 단계 4: 슬라이드 반복 생성 및 내용 채우기 ---
    num_slides_to_generate = len(presentation_outline['slides'])
    logging.info(f"총 {num_slides_to_generate}개의 슬라이드 생성 예정.")

    for i, slide_data in enumerate(presentation_outline['slides']):
        logging.info(f"슬라이드 {i + 1}/{num_slides_to_generate} 생성 중: '{slide_data.get('title', '제목 없음')}'")

        # 적절한 레이아웃 선택
        if i == 0:
            selected_layout = title_layout
        else:
            selected_layout = content_layout

        # 새 슬라이드 추가
        try:
            slide = prs.slides.add_slide(selected_layout)
        except Exception as e:
            logging.error(f"슬라이드 {i + 1} 추가 중 오류 발생: {e}")
            continue # 다음 슬라이드로 진행

        # 상세 내용 생성
        slide_content = generate_slide_content(slide_data['title'], user_prompt)
        if not slide_content:
            logging.warning(f"슬라이드 {i + 1}의 상세 내용을 생성하지 못했습니다. 개요 정보로 대체합니다.")
            # 상세 내용 생성 실패 시, 개요 정보 사용 (대체 로직)
            slide_content = {
                'slide_title': slide_data.get('title', '제목 없음'),
                'bullet_points': [slide_data.get('summary', '내용 없음')]
            }

        # 개체 틀(Placeholder) 식별 및 내용 채우기
        try:
            # 제목 개체 틀 채우기
            if slide.shapes.title:
                title_placeholder = slide.shapes.title
                title_placeholder.text = slide_content.get('slide_title', slide_data.get('title', '')) # 상세 제목 우선, 없으면 개요 제목 사용
                logging.debug(f"  슬라이드 {i + 1}: 제목 채우기 완료 - '{title_placeholder.text}'")
            else:
                logging.warning(f"  슬라이드 {i + 1}: 선택된 레이아웃에 제목 개체 틀(title)이 없습니다.")

            # 내용(본문) 개체 틀 식별 및 채우기
            #!!! 중요: 내용 개체 틀의 인덱스(예: 1)는 사용자의 레이아웃에 따라 다릅니다.!!!
            # 사용자의 'content_layout'에 있는 내용 개체 틀의 실제 인덱스(idx)를 확인해야 합니다.
            # 일반적인 '제목 및 내용' 레이아웃은 idx=1을 사용하지만, 사용자 정의 레이아웃은 다를 수 있습니다.
            # 아래 주석 처리된 코드를 사용하여 특정 슬라이드의 개체 틀 정보를 확인하세요.
            # --------------------------------------------------------------------------
            # 특정 슬라이드의 개체 틀 정보 출력 (사용자 확인용 - 슬라이드 추가 후 실행)
            # print(f"\n슬라이드 {i + 1}의 개체 틀 정보:")
            # for shape in slide.placeholders:
            #     print(f"  인덱스(idx): {shape.placeholder_format.idx}, 이름: {shape.name}, 유형: {shape.placeholder_format.type}")
            # --------------------------------------------------------------------------
            body_placeholder_idx = 1 # 예시 인덱스, 사용자가 수정해야 함
            body_placeholder = None
            try:
                # idx를 사용하여 개체 틀 찾기 (가장 안정적인 방법) [16]
                body_placeholder = slide.placeholders[body_placeholder_idx]
            except KeyError:
                 logging.warning(f"  슬라이드 {i + 1}: 인덱스 {body_placeholder_idx}에 해당하는 내용 개체 틀을 찾을 수 없습니다. 다른 개체 틀을 시도합니다.")
                 # 대체 로직: 이름이나 유형으로 찾거나, 첫 번째 비-제목 개체 틀 사용 시도 (덜 안정적)
                 for ph in slide.placeholders:
                     if ph.placeholder_format.idx!= 0: # 제목(idx=0)이 아닌 첫 번째 개체 틀
                         body_placeholder = ph
                         logging.info(f"  대체 내용 개체 틀 발견: idx={ph.placeholder_format.idx}, name='{ph.name}'")
                         break


            if body_placeholder and body_placeholder.has_text_frame:
                tf = body_placeholder.text_frame
                tf.clear() # 기존 텍스트 삭제 (선택적)
                # tf.text = "" # 첫 번째 단락을 비우는 다른 방법

                bullet_points = slide_content.get('bullet_points',)
                if not bullet_points:
                     logging.warning(f"  슬라이드 {i + 1}: 생성된 글머리 기호 내용이 없습니다.")
                     # 내용이 없을 경우 기본 텍스트 추가 (선택적)
                     p = tf.add_paragraph()
                     p.text = "(내용 생성 실패 또는 내용 없음)"
                     p.level = 0 
                else:
                    for point in bullet_points:
                        p = tf.add_paragraph()
                        p.text = point.strip() # 앞뒤 공백 제거
                        # level=0은 기본 텍스트, level=1부터 글머리 기호 시작 (템플릿 마스터 설정에 따라 다름) [21]
                        p.level = 1 
                logging.debug(f"  슬라이드 {i + 1}: 내용 개체 틀(idx={body_placeholder.placeholder_format.idx}) 채우기 완료 ({len(bullet_points)}개 항목).")
            elif body_placeholder:
                logging.warning(f"  슬라이드 {i + 1}: 내용 개체 틀(idx={body_placeholder.placeholder_format.idx})에 텍스트 프레임이 없습니다 (예: 그림 개체 틀).")
            else:
                 logging.warning(f"  슬라이드 {i + 1}: 내용 개체 틀을 찾지 못하여 내용을 채울 수 없습니다.")

        except AttributeError as e:
            logging.error(f"  슬라이드 {i + 1} 개체 틀 접근 오류: {e}. 레이아웃 인덱스나 개체 틀 인덱스가 올바른지 확인하세요.")
        except Exception as e:
            logging.error(f"  슬라이드 {i + 1} 내용 채우기 중 예상치 못한 오류 발생: {e}")


    # --- 단계 5: 프레젠테이션 저장 ---
    try:
        prs.save(output_filename)
        logging.info(f"프레젠테이션 저장 완료: {output_filename}")
    except Exception as e:
        logging.error(f"프레젠테이션 파일 저장 중 오류 발생: {e}")

In [21]:
from pptx import Presentation

# 사용자 입력 받기
user_topic = "openai 어필을 위한 프레젠테이션"
template_file = r"C:\Users\SSAFY\Downloads\SSAFY Field Trip 활동 보고서 B101.pptx"
output_file = "sample.pptx"

# 입력 값 검증 (기본)
if not user_topic:
    print("오류: 프레젠테이션 주제를 입력해야 합니다.")
elif not template_file.lower().endswith('.pptx'):
    print("오류: 템플릿 파일은.pptx 형식이어야 합니다.")
elif not output_file.lower().endswith('.pptx'):
    print("오류: 출력 파일 이름은.pptx로 끝나야 합니다.")
else:
    # 메인 함수 호출 및 오류 처리
    try:
        create_presentation_from_natural_language(user_topic, template_file, output_file)
        print("-----------------------------------------")
        print("프레젠테이션 생성이 완료되었습니다.")
        print(f"결과 파일: {output_file}")
        print("-----------------------------------------")
    except FileNotFoundError:
            print(f"오류: 템플릿 파일을 찾을 수 없습니다 - {template_file}")
    except AuthenticationError:
            print("오류: OpenAI API 인증에 실패했습니다..env 파일에 유효한 API 키가 있는지 확인하세요.")
    except RateLimitError:
            print("오류: OpenAI API 사용량 제한에 도달했습니다. 잠시 후 다시 시도하거나 사용량 한도를 확인하세요.")
    except APIError as e:
            print(f"오류: OpenAI API 통신 중 오류가 발생했습니다 - {e}")
    except json.JSONDecodeError:
            print("오류: OpenAI API로부터 받은 응답을 처리하는 중 오류가 발생했습니다 (JSON 형식 오류).")
    except Exception as e:
            print(f"오류: 예상치 못한 오류가 발생했습니다 - {e}")

2025-04-22 12:49:13,639 - INFO - 프레젠테이션 생성 프로세스 시작.
2025-04-22 12:49:13,639 - INFO - 프레젠테이션 개요 생성 시작: 주제='openai 어필을 위한 프레젠테이션'
2025-04-22 12:49:18,494 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-22 12:49:18,496 - INFO - 프레젠테이션 개요 생성 완료.
2025-04-22 12:49:18,519 - INFO - 템플릿 로드 성공: C:\Users\SSAFY\Downloads\SSAFY Field Trip 활동 보고서 B101.pptx
2025-04-22 12:49:18,520 - INFO - 사용할 레이아웃 식별: 제목(0), 내용(1)
2025-04-22 12:49:18,521 - INFO - 총 8개의 슬라이드 생성 예정.
2025-04-22 12:49:18,521 - INFO - 슬라이드 1/8 생성 중: 'Introduction to OpenAI'
2025-04-22 12:49:18,522 - INFO - 슬라이드 내용 생성 시작: 제목='Introduction to OpenAI'
2025-04-22 12:49:20,102 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-22 12:49:20,106 - INFO - 슬라이드 내용 생성 완료: 제목='Introduction to OpenAI'
2025-04-22 12:49:20,107 - WARNING -   슬라이드 1: 선택된 레이아웃에 제목 개체 틀(title)이 없습니다.
2025-04-22 12:49:20,110 - WARNING -   슬라이드 1: 인덱스 1에 해당하는 내용 개체 틀을 찾을 수 없습니다. 다른

-----------------------------------------
프레젠테이션 생성이 완료되었습니다.
결과 파일: sample.pptx
-----------------------------------------
